In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',]
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6)
})

output_path = rd.rootdir/'output'/'fig_overview'
cache_path = rd.rootdir/'output'/'fig_overview'/'data.gzip'

# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)


In [ ]:
# Create dicts to specify colors/markers
metadata['miR_'] = (metadata['miR'].isin(['none','na'])).map({True: '–', False:'+'})
metadata['ts_'] = (metadata['ts'].isin(['none','na'])).map({True: '–', False:'+'})
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color'] # colored by design
construct_markers = metadata_dict['markers']

# Create unified color palette for most tuning
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='miR'), 'color'] = base.colors['purple']
metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR'].isin(['miR.FF5','miRE.FF5'])), 'color'] = metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR'].isin(['miR.FF5','miRE.FF5'])), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'color'] = base.colors['blue']
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF6x1','FF6x2','FF6x4'])), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF6x1','FF6x2','FF6x4'])), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF4x1','FF4x2','FF4x4'])), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF4x1','FF4x2','FF4x4'])), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF3x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF3x1'), 'color'].apply(base.get_light_color).apply(base.get_light_color)

metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miR.FF5'), 'color'] = base.colors['green']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miRE.FF5'), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miR.FF4'), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miRE.FF4'), 'color'] = base.colors['teal']

metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['miR'].isin(['miR.FF4','miRE.FF4'])), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['miR'].isin(['miR.FF4','miRE.FF4'])), 'color'].apply(base.get_light_color)

metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==2)), 'color'] = metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==2)), 'color'].apply(base.get_light_color)
metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==4)), 'color'] = metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==4)), 'color'].apply(base.get_dark_color)

metadata2.loc[(metadata2['ts']=='FF6x3.4x1'), 'color'] = metadata2.loc[(metadata2['ts']=='FF6x3.4x1'), 'color'].apply(base.get_light_color)

# markers
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'markers'] = 'D'
metadata2.loc[(metadata2['miR'].isin(['miRE.FF5','miRE.FF4'])), 'markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='CAG'), 'markers'] = 'D'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EFS'), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='hPGK'), 'markers'] = 'v'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'linestyle'] = '--'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
main_palette = metadata_dict2['color']
main_markers = metadata_dict2['markers']

In [ ]:
# Create color palette for comparing promoters
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EF1a'), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='CAG'), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EFS'), 'color'] = base.colors['red']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='hPGK'), 'color'] = base.colors['green']

metadata2.loc[(metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EF1a'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EF1a'), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EFS'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EFS'), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='hPGK'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='hPGK'), 'color'].apply(base.get_dark_color).apply(base.get_dark_color)

# markers
metadata2['markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
promoter_palette = metadata_dict2['color']
promoter_markers = metadata_dict2['markers']

In [ ]:
# Gate cells
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]

In [ ]:
### Bin by marker quantiles
by = ['construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])


### Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')


### Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)


### Compute slope for all constructs
fits = stats_quantiles.groupby(['construct','exp','biorep'])[stats_quantiles.columns].apply(base.get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']

sns.set_style('ticks')
sns.set_context('paper')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': base_size, 'ytick.labelsize': base_size,
                     'pdf.fonttype': 42, 'ytick.major.size': 3, 'xtick.major.size': 3})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,4.375))
fig_gridspec = matplotlib.gridspec.GridSpec(2, 3, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[1,1], width_ratios=[1,1,1])
subfigures = {
    'E': fig.add_subfigure(fig_gridspec[1,1]),
    'F': fig.add_subfigure(fig_gridspec[1,2]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

fig_path = output_path/'fig_overview.pdf'
fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.4, top=0.3, right=0)
axes = subfig.subplots(1,2, sharey=True, gridspec_kw=dict(width_ratios=(1,0.3)))

xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miR.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep) & (df_quantiles['exp']!='exp90.4')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=base.font_sizes['base_size'])
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
#sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
#             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='',)
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.3, bottom=0.4, top=0.3, right=0)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.45))

ylim = (2e1,1e5)

# stat gmean
plot_df = stats[(((stats['miR']=='miR.FF4') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]

ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(['base','unreg.','circuit'], rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1))
ax.set_xticklabels(['base','unreg.','circuit'], rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# slope
plot_df2 = fits[(((fits['miR']=='miR.FF4') & (fits['group']=='controller') & (fits['design']==1) & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (fits['group']=='base')) &
                         (fits['promoter']=='EF1a')]
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['ts'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0','','','','1'])
ax.set_xticklabels(['base','unreg.','circuit'], rotation=45, ha='right')
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(fig_path))